# Adattisztítás és integráció

A valós adathalmazok gyakran zajosak, hiányosak, avagy éppen redundáns információt vagy duplikátum egyedeket tartalmaznak. Ezért a tudásfeltárás folyamatában az adattisztítás és adatintegrálással kezdődik.

Az adattisztítás szerepe javítani az adatok minőségén azáltal, hogy kiszűri és eltávolítja az adatokban fellépő hibákat és inkonzisztenciákat.

Az [adattisztítás](https://hu.wikipedia.org/wiki/Adattiszt%C3%ADtás) során:
"
- felmérjük a hibákat
	- ellenőrizzük az adatfájl szerkezeti épségét
	- a zajt, felesleges információt tartalmazó mezőket javítjuk
	- felmérjük a hiányzó értékeket és amennyiben lehet ezeket pótoljuk
	- felmérjük az adatközlési és adatbeviteli hibákat
		- megvizsgáljuk az egyes változók eloszlását
			- az eloszlások szélein elhelyezkedő extrém értékeket ellenőrizzük
			- felmérjük, hogy az eloszlások megfelelnek-e az előzetes elvárásainknak, vannak-e nem várt sűrűsödések, ritkulások egyes értéktartományokban (például durva kerekítés vagy eltérő mértékegység használata az adatszolgáltatók egy részénél)
		-  megvizsgáljuk, hogy a változók közötti triviális összefüggések teljesülnek-e
-  a hibásnak tűnő adatokat felülvizsgáljuk, javítjuk 
".


In [5]:
from google.colab import files
uploaded = files.upload()

Saving egyetemek.txt to egyetemek.txt


In [6]:
import pandas as pd
import numpy as np

In [7]:
egyetemi_varosok = []
with open('egyetemek.txt') as file:
     for line in file:
         if '[edit]' in line:
             state = line
         else:
             egyetemi_varosok.append((state, line))

In [8]:
egyetemi_varosok[:10]

[('Alabama[edit]\n', 'Auburn (Auburn University)[1]\n'),
 ('Alabama[edit]\n', 'Florence (University of North Alabama)\n'),
 ('Alabama[edit]\n', 'Jacksonville (Jacksonville State University)[2]\n'),
 ('Alabama[edit]\n', 'Livingston (University of West Alabama)[2]\n'),
 ('Alabama[edit]\n', 'Montevallo (University of Montevallo)[2]\n'),
 ('Alabama[edit]\n', 'Troy (Troy University)[2]\n'),
 ('Alabama[edit]\n',
  'Tuscaloosa (University of Alabama, Stillman College, Shelton State)[3][4]\n'),
 ('Alabama[edit]\n', 'Tuskegee (Tuskegee University)[5]\n'),
 ('Alaska[edit]\n', 'Fairbanks (University of Alaska Fairbanks)[2]\n'),
 ('Arizona[edit]\n', 'Flagstaff (Northern Arizona University)[6]\n')]

In [9]:
varosok = pd.DataFrame(egyetemi_varosok,
                        columns=['State', 'City'])

varosok.head()

,State,City
0,Alabama[edit]\n,Auburn (Auburn University)[1]\n
1,Alabama[edit]\n,Florence (University of North Alabama)\n
2,Alabama[edit]\n,Jacksonville (Jacksonville State University)[2]\n
3,Alabama[edit]\n,Livingston (University of West Alabama)[2]\n
4,Alabama[edit]\n,Montevallo (University of Montevallo)[2]\n


In [11]:
def get_state_city(item):
    if ' (' in item:
        return item[:item.find(' (')]
    elif '[' in item:
        return item[:item.find('[')]
    else:
        return item

In [12]:
varosok = varosok.applymap(get_state_city)
varosok

,State,City
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
514,Wisconsin,Stevens Point
515,Wisconsin,Waukesha
516,Wisconsin,Whitewater
517,Wyoming,Laramie


In [13]:
allamok = varosok.groupby("State").size().reset_index(name="Counts")
allamok



,State,Counts
0,123456,1
1,Alabama,8
2,Alaska,1
3,Arizona,3
4,Arkansas,8
5,California,26
6,Colorado,9
7,Connecticut,7
8,Delaware,2
9,Florida,12


In [14]:
allamok[allamok["Counts"] == allamok["Counts"].max()]

,State,Counts
38,Pennsylvania,43


In [15]:
allamok[allamok["Counts"] == allamok["Counts"].min()]

,State,Counts
0,123456,1
2,Alaska,1
11,Hawaii,1
50,Wyoming,1


In [16]:
allamrov = {"Alabama":"AL","Alaska":"AK","Arizona":"AZ","Arkansas":"AR","California":"CA","Colorado":"CO","Connecticut":"CT","Delaware":"DE","Florida":"FL","Georgia":"GA","Hawaii":"HI","Idaho":"ID","Illinois":"IL","Indiana":"IN","Iowa":"IA","Kansas":"KS","Kentucky":"KY","Louisiana":"LA","Maine":"ME","Maryland":"MD","Massachusetts":"MA","Michigan":"MI","Minnesota":"MN","Mississippi":"MS","Missouri":"MO","Montana":"MT","Nebraska":"NE","Nevada":"NV","New Hampshire":"NH","New Jersey":"NJ","New Mexico":"NM","New York":"NY","North Carolina":"NC","North Dakota":"ND","Ohio":"OH","Oklahoma":"OK","Oregon":"OR","Pennsylvania":"PA","Rhode Island":"RI","South Carolina":"SC","South Dakota":"SD","Tennessee":"TN","Texas":"TX","Utah":"UT","Vermont":"VT","Virginia":"VA","Washington":"WA","West Virginia":"WV","Wisconsin":"WI","Wyoming":"WY"}
varosok['abbrev'] = varosok['State'].map(allamrov)
varosok

,State,City,abbrev
0,Alabama,Auburn,AL
1,Alabama,Florence,AL
2,Alabama,Jacksonville,AL
3,Alabama,Livingston,AL
4,Alabama,Montevallo,AL
...,...,...,...
514,Wisconsin,Stevens Point,WI
515,Wisconsin,Waukesha,WI
516,Wisconsin,Whitewater,WI
517,Wyoming,Laramie,WY


In [17]:
allamnep = {"Alabama":4903185,"Alaska":731545,"Arizona":7278717,"Arkansas":3017825,"California":39512223,"Colorado":5758736,"Connecticut":3565287,"Delaware":973764,"Florida":21477737,"Georgia":10617423,"Hawaii":1415872,"Idaho":1787065,"Illinois":12671821,"Indiana":6732219,"Iowa":3155070,"Kansas":2913314,"Kentucky":4467673,"Louisiana":4648794,"Maine":1344212,"Maryland":6045680,"Massachusetts":6949503,"Michigan":9986857,"Minnesota":5639632,"Mississippi":2976149,"Missouri":6137428,"Montana":1068778,"Nebraska":1934408,"Nevada":3080156,"New Hampshire":1359711,"New Jersey":8882190,"New Mexico":2096829,"New York":19453561,"North Carolina":10488084,"North Dakota":762062,"Ohio":11689100,"Oklahoma":3956971,"Oregon":4217737,"Pennsylvania":12801989,"Rhode Island":1059361,"South Carolina":5148714,"South Dakota":884659,"Tennessee":6833174,"Texas":28995881,"Utah":3205958,"Vermont":623989,"Virginia":8535519,"Washington":7614893,"West Virginia":1792147,"Wisconsin":5822434,"Wyoming":578759}
varosok['population'] = varosok['State'].map(allamnep)
varosok

,State,City,abbrev,population
0,Alabama,Auburn,AL,4903185.0
1,Alabama,Florence,AL,4903185.0
2,Alabama,Jacksonville,AL,4903185.0
3,Alabama,Livingston,AL,4903185.0
4,Alabama,Montevallo,AL,4903185.0
...,...,...,...,...
514,Wisconsin,Stevens Point,WI,5822434.0
515,Wisconsin,Waukesha,WI,5822434.0
516,Wisconsin,Whitewater,WI,5822434.0
517,Wyoming,Laramie,WY,578759.0


In [18]:
allamok2 = varosok.groupby(["State","abbrev","population"]).size().reset_index(name="Counts")
allamok2["capita"] = round((allamok2.population/allamok2.Counts),2)
allamok2

,State,abbrev,population,Counts,capita
0,Alabama,AL,4903185.0,8,612898.12
1,Alaska,AK,731545.0,1,731545.00
2,Arizona,AZ,7278717.0,3,2426239.00
3,Arkansas,AR,3017825.0,8,377228.12
4,California,CA,39512223.0,26,1519700.88
5,Colorado,CO,5758736.0,9,639859.56
6,Connecticut,CT,3565287.0,7,509326.71
7,Delaware,DE,973764.0,2,486882.00
8,Florida,FL,21477737.0,12,1789811.42
9,Georgia,GA,10617423.0,16,663588.94


In [27]:
allamter = {"Alabama":135767,"Alaska":135767,"Arizona":295234,"Arkansas":137732,"California":423967,"Colorado":269601,"Connecticut":14357,"Delaware":6446,"Florida":170312,"Georgia":153910,"Hawaii":28313,"Idaho":216443,"Illinois":149995,"Indiana":94326,"Iowa":145746,"Kansas":213100,"Kentucky":104656,"Louisiana":135659,"Maine":91633,"Maryland":32131,"Massachusetts":27336,"Michigan":250487,"Minnesota":225163,"Mississippi":125438,"Missouri":180540,"Montana":380831,"Nebraska":200330,"Nevada":286380,"New Hampshire":24214,"New Jersey":22591,"New Mexico":314917,"New York":141297,"North Carolina":139391,"North Dakota":183108,"Ohio":116098,"Oklahoma":181037,"Oregon":254799,"Pennsylvania":119280,"Rhode Island":4001,"South Carolina":82933,"South Dakota":199729,"Tennessee":109153,"Texas":695662,"Utah":219882,"Vermont":24906,"Virginia":110787,"Washington":184661,"West Virginia":62756,"Wisconsin":169635,"Wyoming":253335}
varosok['area'] = varosok['State'].map(allamter)


In [21]:
allamok3 = varosok.groupby(["State","abbrev","population","area"]).size().reset_index(name="Counts")
allamok3["capita"] = round((allamok3.population/allamok3.Counts),2)
allamok3["capita/km2"] = round((allamok3.area/allamok3.Counts),2)
allamok3

,State,abbrev,population,area,Counts,capita,capita/km2
0,Alabama,AL,4903185.0,135767.0,8,612898.12,16970.88
1,Alaska,AK,731545.0,135767.0,1,731545.00,135767.00
2,Arizona,AZ,7278717.0,295234.0,3,2426239.00,98411.33
3,Arkansas,AR,3017825.0,137732.0,8,377228.12,17216.50
4,California,CA,39512223.0,423967.0,26,1519700.88,16306.42
5,Colorado,CO,5758736.0,269601.0,9,639859.56,29955.67
6,Connecticut,CT,3565287.0,14357.0,7,509326.71,2051.00
7,Delaware,DE,973764.0,6446.0,2,486882.00,3223.00
8,Florida,FL,21477737.0,170312.0,12,1789811.42,14192.67
9,Georgia,GA,10617423.0,153910.0,16,663588.94,9619.38


# Feladatok

1. Az `egyetemek.txt` fájlból szűrjük ki az államokat és azon belül a városokat, melyben egyetemek találhatóak. Ha vannak duplikátumok, helytelen adatok (pl. számokat tartalmazó államnév), ezeket javítsuk. Vizsgáljuk meg az egyetemek eloszlását államok szerint. Melyik államban van a legtöbb, legkevesebb egyetem?

2. Bővítsük ki az adatbázisunkat egy oszloppal, mely tartalmazza az államok rövidítését is (pl. Texas - TX, California - CA stb.).  [Forrás](https://en.wikipedia.org/wiki/List_of_U.S._state_abbreviations).

3. Az adatbázist integráljuk a [List of states and territories of the United States by population](https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States_by_population) linken szereplő népszámlálási adatokkal és számoljuk ki államonként hány főre jut egy egyetemi város.

4. Hasonlóan, a [List of U.S. states and territories by area](https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_area) linken szereplő területi adatok integrálásával, számoljuk ki államonként átlagban hány négyzetkilométerre jut egy egyetemi város.

5. A [példa](https://datasciencechalktalk.com/2019/09/28/analyzing-u-s-exports-with-plotly/), a kivitelt, exportmennyiséget ábrázolja az Amerikai Egyesült Államok térképen, államonként lebontva ezt. Készítsünk hasonló ábrákat az egyetemek abszolút, lakoság és terület szerinti eloszlásáról is. Az ábrákat exportáljuk kép formájában.
![](https://i.ibb.co/s1zdnLY/mapplot.png)

  Ábra 1. [Térképen való ábrázolás](https://plotly.com/python/maps/) Plotly segítségével. [Forrás](https://datasciencechalktalk.com/2019/09/28/analyzing-u-s-exports-with-plotly/).

6. Számoljuk ki, hány egyetem van városonként, majd összesítve államonként. Ehhez az `egyetemek.txt` fájlban miután megkapunk egy várost, az utána következő kerek zárójelek között megszámoljuk, hány vesszővel elválasztott karakterlánc található. 
Pl. a következő sorban:	`Claremont (Claremont McKenna College, Pomona College, Harvey Mudd College, Scripps College, Pitzer College, Keck Graduate Institute, Claremont Graduate University)[5]` -> 6 vessző van a kerek zárójelek között, tehát 7 egyetem van a városban.    
Melyik államban és melyik városban van a legtöbb egyetem?


In [24]:
import plotly.graph_objects as go
import pandas as pd

fig = go.Figure(data=go.Choropleth(
    locations=allamok3["abbrev"],
    z = allamok3["Counts"].astype(float),
    locationmode = "USA-states",
    colorscale = "Reds",
    colorbar_title = "Egyetemi varosok szama",
))
fig.update_layout(
    title_text = "Egyetemek szama allamonket az USA-ban",
    geo_scope="usa",
)
fig.show()


In [25]:
fig = go.Figure(data=go.Choropleth(
    locations=allamok3["abbrev"],
    z = allamok3["capita/km2"].astype(float),
    locationmode = "USA-states",
    colorscale = "Reds",
    colorbar_title = "Hany km2 -re",
))
fig.update_layout(
    title_text = "Allamonkent hany negyzetkilometerre jut egy egyetem",
    geo_scope="usa",
)
fig.show()

In [26]:
fig = go.Figure(data=go.Choropleth(
    locations=allamok3["abbrev"],
    z = allamok3["capita"].astype(float),
    locationmode = "USA-states",
    colorscale = "Reds",
    colorbar_title = "Hany fore",
))
fig.update_layout(
    title_text = "Allamonkent hany fore jut egy egyetem",
    geo_scope="usa",
)
fig.show()